In [11]:
import requests
!pip install bs4
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import pandas as pd
import json

In [12]:
url = 'https://mybook.ru/catalog/books/'
page = requests.get(url)
page

<Response [200]>

In [13]:
soup = BeautifulSoup(page.text, 'html')
print(soup.prettify())

<!DOCTYPE html>
<html lang="ru">
 <head>
  <link href="https://i3.mybook.io/next/favicons/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="https://i2.mybook.io/next/favicons/favicon-48x48.png" rel="icon" sizes="48x48" type="image/png"/>
  <link href="https://i3.mybook.io/next/favicons/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="https://i2.mybook.io/next/favicons/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="https://i1.mybook.io/static/rebook/manifest/yandex.json" rel="manifest"/>
  <link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
  <meta charset="utf-8"/>
  <meta content="#fec41f" name="msapplication-TileColor"/>
  <meta content="https://i1.mybook.io/next/favicons/mstile-144x144.png" name="msapplication-TileImage"/>
  <meta content="/next/favicons/browserconfig.xml" name="msapplication-config"/>
  <meta content="#fec41f" name="theme-color"/>
  <meta content="app-id=556540

In [14]:
# <a href="/author/aleksej-goncharov/uspeh-ili-uspet/">
unique_urls = set()
for i in tqdm(range(5)): #!!!потом вместо 5 нужно ввести колличество страниц которое будем парсить!!!
    url = 'https://mybook.ru/catalog/books/?page=' + str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')
    for link in soup.find_all('a'):
        a = link.get('href')
        if a is not None and a.count('/') == 4 and 'author' in a:
            unique_urls.add('https://mybook.ru/' + a)
    sleep(1)
for url in unique_urls:
    print(url)

100%|██████████| 5/5 [00:11<00:00,  2.24s/it]

https://mybook.ru//author/rebekka-yarros-2/oniksovyj-shtorm/
https://mybook.ru//author/aleks-mihaelides/bezmolvnyj-pacient/
https://mybook.ru//author/haled-hossejni/begushij-za-vetrom/
https://mybook.ru//author/geri-chepmen/pyat-yazykov-lyubvi-kak-vyrazit-lyubov-vashemu-spu/
https://mybook.ru//author/ivan-sergeevich-turgenev/otcy-i-deti-1/
https://mybook.ru//author/rina-skih/nervnye-kletki-s-zapasom-3/
https://mybook.ru//author/blejk-pirs/sled-smerti/
https://mybook.ru//author/geri-chepmen/pyat-yazykov-lyubvi-kak-vyrazit-lyubov-vashemu-s-5/
https://mybook.ru//author/nadezhda-reznikova/zhivaya-psihologiya-prostye-shagi-navstrechu-k-seb/
https://mybook.ru//author/robert-grin/48-zakonov-vlasti/
https://mybook.ru//author/aleksej-goncharov/uspeh-ili-uspet/
https://mybook.ru//author/dzhon-keho/podsoznanie-mozhet-vsyo/
https://mybook.ru//author/garet-braun/kniga-dverej/
https://mybook.ru//author/fredrik-bakman/vtoraya-zhizn-uve/
https://mybook.ru//author/robert-kijosaki/bogatyj-papa-bednyj-pa

In [15]:
book_data = []
for url in tqdm(unique_urls):
    try:
        page = requests.get(url)
        if page.status_code == 200:
            soup = BeautifulSoup(page.text, 'html.parser')
            script_tag = soup.find('script', type='application/ld+json')
            json_data = json.loads(script_tag.string) if script_tag else {}

            name = json_data.get('name')
            author = json_data.get('author', {}).get('name')
            rating = json_data.get('aggregateRating', {}).get('ratingValue')
            review_count = json_data.get('aggregateRating', {}).get('ratingCount')
            date_published_json = json_data.get('datePublished')
            number_of_pages_json = json_data.get('numberOfPages')
            genre_json = json_data.get('genre')
            description_json = json_data.get('description')
            language_json = json_data.get('inLanguage')
            age_restriction_json = json_data.get('audience', {}).get('audienceType')
            book_format_json = json_data.get('bookFormat')
            time_required_json = json_data.get('timeRequired')
            publisher_json = json_data.get('publisher', {}).get('name')

            time_info_html = None
            age_restriction_html = None
            number_of_pages_html = None
            reading_time = None
            listening_time = None

            info_div = soup.find('div', class_='ant-col sc-1c0xbiw-9 eSjGMZ')
            if info_div:
                p_tags = info_div.find_all('p', class_='lnjchu-1 dPgoNf')
                if len(p_tags) > 0:
                    time_info_html = p_tags[0].text.strip()
                    if 'печатных страниц' in time_info_html:
                        number_of_pages_html = time_info_html.split(' ')[0]
                    elif 'час.' in time_info_html or 'ч' in time_info_html:
                        listening_time = time_info_html
                if len(p_tags) > 3:
                    age_restriction_html = p_tags[3].text.strip()
                if number_of_pages_html and len(p_tags) > 1:
                     if 'Время чтения' in p_tags[1].text.strip():
                        reading_time = p_tags[1].text.strip().replace('Время чтения ≈', '').strip()


            name = name if name else None
            author = author if author else None
            rating = rating if rating else None
            review_count = review_count if review_count else None
            date_published = date_published_json if date_published_json else None
            number_of_pages = number_of_pages_html if number_of_pages_html else number_of_pages_json
            genre = genre_json if genre_json else None
            description = description_json if description_json else None
            language = language_json if language_json else None
            age_restriction = age_restriction_html if age_restriction_html else age_restriction_json
            book_format = book_format_json if book_format_json else None
            publisher = publisher_json if publisher_json else None

            book_data.append({
                'Название': name,
                'Автор': author,
                'Средняя оценка': rating,
                'Количество отзывов': review_count,
                'Год выпуска': date_published,
                'Количество страниц': number_of_pages,
                'Жанр': genre,
                'Описание': description,
                'Язык': language,
                'Возрастное ограничение': age_restriction,
                'Формат книги': book_format,
                'Время чтения': reading_time,
                'Время слушания': listening_time,
                'Издатель': publisher,
                'URL': url
            })

        else:
            print(f"Не удалось получить страницу {url}. Код статуса: {page.status_code}")
    except Exception as e:
        print(f"Произошла ошибка при обработке {url}: {e}")

100%|██████████| 40/40 [01:15<00:00,  1.89s/it]


In [16]:
book_df = pd.DataFrame(book_data)
display(book_df.head())
display(book_df.tail())

,Название,Автор,Средняя оценка,Количество отзывов,Год выпуска,Количество страниц,Жанр,Описание,Язык,Возрастное ограничение,Формат книги,Время чтения,Время слушания,Издатель,URL
0,Ониксовый шторм,Ребекка Яррос,4.77,401,2025-08-15,701,"Боевое фэнтези, Любовное фэнтези, Зарубежное ф...","Третья книга сенсационной серии «Эмпирей», взо...",ru,18+,None,None,701 печатная страница,Кислород,https://mybook.ru//author/rebekka-yarros-2/oni...
1,Безмолвный пациент,Алекс Михаэлидес,4.60,14764,2019-02-27,252,"Триллеры, Современные детективы, Зарубежные де...",Жизнь Алисии Беренсон кажется идеальной. Извес...,ru,18+,None,None,252 печатные страницы,Эксмо,https://mybook.ru//author/aleks-mihaelides/bez...
2,Бегущий за ветром,Халед Хоссейни,4.71,7221,2011-09-28,286,Современная зарубежная литература,«Бегущий за ветром» — это захватывающий и прон...,ru,16+,None,8ч,None,Фантом Пресс,https://mybook.ru//author/haled-hossejni/begus...
3,Пять языков любви,Гэри Чепмен,4.70,10710,2013-12-16,113,"Семья, брак, сексология, Саморазвитие, личност...",Как выразить любовь вашему спутнику. Любовь мо...,ru,12+,None,3ч,None,Библия для всех,https://mybook.ru//author/geri-chepmen/pyat-ya...
4,Отцы и дети,Иван Тургенев,4.64,8243,2008-11-05,206,"Школьные учебники, Русская классика, Литератур...",В романе «Отцы и дети» отразилась идеологическ...,ru,12+,None,6ч,None,Public Domain,https://mybook.ru//author/ivan-sergeevich-turg...


,Название,Автор,Средняя оценка,Количество отзывов,Год выпуска,Количество страниц,Жанр,Описание,Язык,Возрастное ограничение,Формат книги,Время чтения,Время слушания,Издатель,URL
35,Братья Карамазовы,Федор Достоевский,4.67,4902,2008-11-03,1 008,"Русская классика, Литература 19 века","Самый сложный, самый многоуровневый и неоднозн...",ru,12+,None,26ч,None,Public Domain,https://mybook.ru//author/fedor-mihajlovich-do...
36,Сто лет одиночества,Габриэль Гарсиа Маркес,4.23,6130,2012-11-16,419,"Зарубежная классика, Литература 20 века",Все книги Габриэля Гарсии Маркеса обладают осо...,ru,12+,None,11ч,None,Издательство АСТ,https://mybook.ru//author/gabriel-markes/sto-l...
37,Преступление и наказание,Федор Достоевский,4.64,12489,2008-11-03,598,"Школьные учебники, Литература 19 века, Русская...",«Преступление и наказание» — классический псих...,ru,12+,None,15ч,None,Public Domain,https://mybook.ru//author/fedor-mihajlovich-do...
38,"Бражники и блудницы. Как жили, любили и умирал...",Максим Жегалин,4.55,51,2025-07-21,316,Биографии и мемуары,Александр Блок гуляет по окраинам Петербурга и...,ru,18+,None,8ч,None,Individuum / Popcorn books,https://mybook.ru//author/maksim-zhegalin/braz...
39,Громов: Хозяин теней – 4,Карина Демина,4.82,453,2025-07-29,302,"Попаданцы, Героическое фэнтези",Официально род Громовых оборвался. Вот только ...,ru,18+,None,None,302 печатные страницы,автор,https://mybook.ru//author/karina-demina/gromov...


In [17]:
book_df.to_csv('book_data.csv', index=False)
print("Датасет сохранен в файл 'book_data.csv'")

Датасет сохранен в файл 'book_data.csv'
